<a href="https://colab.research.google.com/github/tchumphrey65/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:5 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:13 https://developer.download.nvidia.com/compute/

In [2]:
from pyspark.sql.types import StructType,StructField 
from pyspark.sql.types import StringType, IntegerType, ArrayType

In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [5]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Outdoors_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   18446823|R35T75OLUGHL5C|B000NV6H94|     110804376|Stearns Youth Boa...|        Outdoors|          4|            0|          0|   N|                Y|          Four Stars|          GOOD VALUE| 2015-08-31|
|         US|   13724367|R2BV735O46BN33|B000IN0W3Y|     624096774|Primal Wear Men's...|        Outdoors|          5|    

In [6]:
# Create the customers_table DataFrame
customers_table = df.select(["customer_id"])

customers_df = customers_table.groupby("customer_id").agg({"customer_id":"count"}).withColumnRenamed("count(customer_id)", "customer_count")

customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   43679767|             1|
|   32024654|             1|
|   52913169|             1|
|   24297214|             1|
|   26096454|             1|
|   38247118|             1|
|   44009906|             1|
|    1753876|             1|
|   26195644|             1|
|   23042837|             1|
|   11487525|             2|
|   28258386|             1|
|   42560427|             2|
|   24540309|             1|
|    1967239|             1|
|   14217455|             1|
|   20289484|            26|
|   47103434|             1|
|   28638887|             1|
|     740678|             1|
+-----------+--------------+
only showing top 20 rows



In [7]:
# Create the products_table DataFrame and drop duplicates. 
# products_df = df.select([]).drop_duplicates()
products_df = df.select(["product_id","product_title"]).drop_duplicates(["product_id"])

products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|6040161299|Santa Cruz Scream...|
|B0000AXTWB|Kwik Tek SF-1C Su...|
|B0000BYDQX|Bellows Foot Pump...|
|B0000DYNXN|DIN &amp; Yoke Ta...|
|B0000E66XJ|Black Diamond Zio...|
|B00012O8B4|Razor E100 Electr...|
|B00029NCOK|ATV Logic ATVHP-B...|
|B0002BJZ6W|Pro XL-C Corragat...|
|B0002CTKO8|Heat Factory Flee...|
|B0002LKM8W|Weekender Men's T...|
|B0002M9DO0|STABILicers Maxx ...|
|B000650ZMW|Panaracer, 700x32...|
|B00066Z29W|             Fairing|
|B00068NHMO|Marmot Men's Midw...|
|B0006FYKCI|Transpack Edge Is...|
|B0006UOKFA|FireOne Windproof...|
|B0006VSTOW|Flowlab Blue Logo...|
|B0007IS6BA|Columbia Bugaboo ...|
|B0007QCO4M|              Bantam|
|B0007QCOP6|Victorinox Swiss ...|
+----------+--------------------+
only showing top 20 rows



In [9]:
# Create the vine_table. DataFrame
# vine_df = df.select([])
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "Vine", "Verified_purchase"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|Vine|Verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R35T75OLUGHL5C|          4|            0|          0|   N|                Y|
|R2BV735O46BN33|          5|            0|          0|   N|                Y|
|R2NBEUGPQQGXP1|          4|            0|          0|   N|                Y|
|R17LLAOJ8ITK0S|          3|            1|          1|   N|                Y|
|R39PEQBT5ISEF4|          1|            0|          0|   N|                Y|
|R3GNM3SU9VHJFT|          4|            1|          1|   N|                Y|
| R2Y81OP0EK467|          5|            0|          0|   N|                Y|
|R2LFGSI6HAYH5F|          5|            1|          1|   N|                Y|
|R297G6ED1IQO7W|          5|            1|          1|   N|                Y|
| RE27RFC6101N6|          5|            0|          0|   N|     

In [11]:
# Filter Vine_df to retrieve all the rows where the total_votes count is greater than or equal to 20 

filtered_vine_df = vine_df.filter("total_votes >= 20")
filtered_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|Vine|Verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R2O4P39D0ALCQK|          2|           14|         43|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|     

In [13]:
# Create a new DataFrame or table to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
filtered_vine_votes_df = filtered_vine_df.filter("helpful_votes/total_votes>=0.50")
filtered_vine_votes_df.show()
# Add Column to the filtered vine votes dataframe showing helpful votes / total votes calulation
helpful_vine_votes_df = filtered_vine_votes_df.withColumn("helpful_votes/total_votes", filtered_vine_votes_df["helpful_votes"]/filtered_vine_votes_df["total_votes"])
helpful_vine_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|Vine|Verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [19]:
# Create a DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid)
vine_program_paid_df = filtered_vine_votes_df.filter(filtered_vine_votes_df.Vine == "Y")
vine_program_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|Vine|Verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KPC0NBUDASX3|          5|           25|         25|   Y|                N|
|R119P2A95GGXX4|          5|           26|         28|   Y|                N|
|R1HKIRME8AJ89Z|          5|           79|         82|   Y|                N|
|R3FY3GMBGOBR22|          5|           12|         20|   Y|                N|
|R15KH3FBSVYGBU|          5|           32|         37|   Y|                N|
|R22XZEQLVLWW49|          5|           28|         33|   Y|                N|
|R2OIQYVAL9D63R|          4|           59|         65|   Y|                N|
|R1W3KCDHM6BMZT|          4|           37|         38|   Y|                N|
|R3KDMPS96IJ10R|          2|           15|         21|   Y|                N|
|R1XWEI58A7VKLB|          5|          123|        128|   Y|     

In [20]:
vine_program_not_paid_df = filtered_vine_votes_df.filter(filtered_vine_votes_df.Vine == "N")
vine_program_not_paid_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|Vine|Verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R2FP3U4NHNFNL2|          5|           25|         29|   N|                Y|
|R1UUK1977O38MU|          5|           31|         31|   N|                Y|
| RXO216LWUDV6O|          3|           29|         31|   N|                Y|
|R3NMJF7EBMM19V|          3|           26|         27|   N|                Y|
|R2ZY0ZBDUO0XUY|          3|           20|         21|   N|                Y|
| RBDVHX7PMQEKM|          5|           35|         39|   N|                Y|
| RGO3KSDXYWTGU|          4|           26|         26|   N|                Y|
|R1E1PQGHWGO3Q1|          1|           89|         96|   N|                Y|
|R1XOXXUVYZEDJB|          5|           22|         22|   N|                Y|
| R7SHITUU5UG9U|          5|           27|         27|   N|     

In [24]:
#Calculate total number of reviews of filtered datafrom (votes >20 and helpful > 50%) 
total_review_count = filtered_vine_votes_df.count()
total_review_count

39976

In [31]:
# Determine the total number of reviews for paid (Vine program)
vine_program_paid_review_count = vine_program_paid_df.count()
vine_program_paid_review_count

107

In [32]:
# Determine the total number of reviews for unpaid (not a part of Vine program)
vine_program_not_paid_review_count = vine_program_not_paid_df.count()
vine_program_not_paid_review_count

39869

In [35]:
# Determine the number of 5-star reviews for Vine program reviews
paid_five_star_review_count = vine_program_paid_df.filter("star_rating==5").count()
paid_five_star_review_count

56

In [34]:
# Determine the number of 5-star reviews for non Vine program reviews
not_paid_five_star_review_count = vine_program_not_paid_df.filter("star_rating==5").count()
not_paid_five_star_review_count

21005

In [30]:
# Verify the total 5 star reviews matches the sum of the above individual filtered df's
# Determine the total number of 5-star reviews
five_star_reviews = filtered_vine_votes_df.filter("star_rating==5").count()
five_star_reviews

21061

In [36]:
# Determine the percentage of Vine 5 Star reviews

vine_five_star_percent = paid_five_star_review_count / vine_program_paid_review_count
vine_five_star_percent

0.5233644859813084

In [37]:
# Determine the percentage of Non - Vine 5 Star reviews
non_vine_five_star_percent = not_paid_five_star_review_count / vine_program_not_paid_review_count
non_vine_five_star_percent

0.5268504351751988

In [38]:
#Total Percentage 5 Star Reviews

five_star_review_percentage = five_star_reviews / total_review_count
five_star_review_percentage

0.5268411046627977